### Neural Machine Translation using word level language model and embeddings in Keras

In [1]:
import pandas as pd
import numpy as np
import string
from string import digits
import matplotlib.pyplot as plt
%matplotlib inline
import re
from sklearn.cross_validation import train_test_split
# Building a english to french translator

/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [106]:
lines= pd.read_table('fra.txt', names=['eng', 'fr'])

In [26]:
lines = lines[0:50000]

In [27]:
lines.shape

(50000, 2)

In [124]:
lines.sample(10)

,eng,fr
66689,It's crawling with spiders.,Ça grouille d'araignées.
46398,Do you have any aspirin?,As-tu de l'aspirine ?
50074,They're looking for you.,Ils te cherchent.
29658,Can you make it safe?,Pouvez-vous vous en sortir saine et sauve ?
39147,Tom folded his shirts.,Tom a plié ses chemises.
14177,Leave this to me.,Laissez.
75813,"Have a cup of tea, won't you?",Vous prendrez bien une tasse de thé ?
54350,I'm glad to see you here.,Je suis contente de te voir ici.
133010,The president is a down-to-earth kind of man.,Le président est plutôt quelqu'un de terre-à-t...
20122,You're very timid.,Vous êtes fort craintif.


#### Data Cleanup

In [28]:
lines.eng=lines.eng.apply(lambda x: x.lower())
lines.fr=lines.fr.apply(lambda x: x.lower())

In [33]:
# Take the length as 50
lines.eng=lines.eng.apply(lambda x: re.sub("'", '', x)).apply(lambda x: re.sub(",", ' COMMA', x))
lines.fr=lines.fr.apply(lambda x: re.sub("'", '', x)).apply(lambda x: re.sub(",", ' COMMA', x))

In [29]:
exclude = set(string.punctuation)
lines.eng=lines.eng.apply(lambda x: ''.join(ch for ch in x if ch not in exclude))
lines.fr=lines.fr.apply(lambda x: ''.join(ch for ch in x if ch not in exclude))

In [30]:
remove_digits = str.maketrans('', '', digits)
lines.eng=lines.eng.apply(lambda x: x.translate(remove_digits))
lines.fr=lines.fr.apply(lambda x: x.translate(remove_digits))

In [33]:
lines.head()

,eng,fr
0,go,START_ va _END
1,run,START_ cours _END
2,run,START_ courez _END
3,wow,START_ ça alors _END
4,fire,START_ au feu _END


#### Generate synthetic data

In [32]:
lines.fr = lines.fr.apply(lambda x : 'START_ '+ x + ' _END')

In [34]:
all_eng_words=set()
for eng in lines.eng:
    for word in eng.split():
        if word not in all_eng_words:
            all_eng_words.add(word)
    
all_french_words=set()
for fr in lines.fr:
    for word in fr.split():
        if word not in all_french_words:
            all_french_words.add(word)

In [35]:
len(all_eng_words), len(all_french_words)

(6029, 12762)

In [36]:
lenght_list=[]
for l in lines.fr:
    lenght_list.append(len(l.split(' ')))
np.max(lenght_list)

16

In [37]:
lenght_list=[]
for l in lines.eng:
    lenght_list.append(len(l.split(' ')))
np.max(lenght_list)

7

In [38]:
input_words = sorted(list(all_eng_words))
target_words = sorted(list(all_french_words))
num_encoder_tokens = len(all_eng_words)
num_decoder_tokens = len(all_french_words)
# del all_eng_words, all_french_words

In [39]:
input_token_index = dict(
    [(word, i) for i, word in enumerate(input_words)])
target_token_index = dict(
    [(word, i) for i, word in enumerate(target_words)])

In [40]:
len(lines.fr)*16*num_decoder_tokens

10209600000

In [51]:
encoder_input_data = np.zeros(
    (len(lines.eng), 7),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(lines.fr), 16),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(lines.fr), 16, num_decoder_tokens),
    dtype='float32')

In [52]:
for i, (input_text, target_text) in enumerate(zip(lines.eng, lines.fr)):
    for t, word in enumerate(input_text.split()):
        encoder_input_data[i, t] = input_token_index[word]
    for t, word in enumerate(target_text.split()):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t] = target_token_index[word]
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[word]] = 1.

### Build keras encoder-decoder model

In [53]:
embedding_size = 50

In [76]:
from keras.layers import Input, LSTM, Embedding, Dense
from keras.models import Model
from keras.utils import plot_model

#### Encoder model

In [77]:
encoder_inputs = Input(shape=(None,))
en_x=  Embedding(num_encoder_tokens, embedding_size)(encoder_inputs)
encoder = LSTM(50, return_state=True)
encoder_outputs, state_h, state_c = encoder(en_x)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

#### Decoder model

In [78]:
# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None,))

dex=  Embedding(num_decoder_tokens, embedding_size)

final_dex= dex(decoder_inputs)


decoder_lstm = LSTM(50, return_sequences=True, return_state=True)

decoder_outputs, _, _ = decoder_lstm(final_dex,
                                     initial_state=encoder_states)

decoder_dense = Dense(num_decoder_tokens, activation='softmax')

decoder_outputs = decoder_dense(decoder_outputs)

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])


In [79]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_20 (InputLayer)           (None, None)         0                                            
__________________________________________________________________________________________________
input_21 (InputLayer)           (None, None)         0                                            
__________________________________________________________________________________________________
embedding_6 (Embedding)         (None, None, 50)     301450      input_20[0][0]                   
__________________________________________________________________________________________________
embedding_7 (Embedding)         (None, None, 50)     638100      input_21[0][0]                   
__________________________________________________________________________________________________
lstm_6 (LS

#### Fit the model

In [109]:
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=128,
          epochs=20,
          validation_split=0.05)

Train on 47500 samples, validate on 2500 samples
Epoch 1/20
47500/47500 [==============================] - 90s 2ms/step - loss: 0.7957 - acc: 0.2040 - val_loss: 1.3757 - val_acc: 0.1902
Epoch 2/20
47500/47500 [==============================] - 90s 2ms/step - loss: 0.7902 - acc: 0.2052 - val_loss: 1.3847 - val_acc: 0.1892
Epoch 3/20
47500/47500 [==============================] - 90s 2ms/step - loss: 0.7852 - acc: 0.2068 - val_loss: 1.3804 - val_acc: 0.1902
Epoch 4/20
47500/47500 [==============================] - 90s 2ms/step - loss: 0.7801 - acc: 0.2081 - val_loss: 1.3787 - val_acc: 0.1917
Epoch 5/20
47500/47500 [==============================] - 90s 2ms/step - loss: 0.7751 - acc: 0.2094 - val_loss: 1.3744 - val_acc: 0.1927
Epoch 6/20
47500/47500 [==============================] - 91s 2ms/step - loss: 0.7694 - acc: 0.2106 - val_loss: 1.3815 - val_acc: 0.1901
Epoch 7/20
47500/47500 [==============================] - 90s 2ms/step - loss: 0.7620 - acc: 0.2118 - val_loss: 1.3710 - val_acc:

In [110]:
encoder_model = Model(encoder_inputs, encoder_states)
encoder_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_20 (InputLayer)        (None, None)              0         
_________________________________________________________________
embedding_6 (Embedding)      (None, None, 50)          301450    
_________________________________________________________________
lstm_6 (LSTM)                [(None, 50), (None, 50),  20200     
Total params: 321,650
Trainable params: 321,650
Non-trainable params: 0
_________________________________________________________________


#### Create sampling model

In [111]:
decoder_state_input_h = Input(shape=(50,))
decoder_state_input_c = Input(shape=(50,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

final_dex2= dex(decoder_inputs)

decoder_outputs2, state_h2, state_c2 = decoder_lstm(final_dex2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]
decoder_outputs2 = decoder_dense(decoder_outputs2)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2)

# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())


#### Function to generate sequences

In [112]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0] = target_token_index['START_']

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += ' '+sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '_END' or
           len(decoded_sentence) > 52):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # Update states
        states_value = [h, c]

    return decoded_sentence

#### Look at the some translations

In [128]:
for seq_index in [14077,20122,40035,40064, 40056, 40068, 40090, 40095, 40100, 40119, 40131, 40136, 40150, 40153]:
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', lines.eng[seq_index: seq_index + 1])
    print('Decoded sentence:', decoded_sentence)

-
Input sentence: 14077    It's almost time.
Name: eng, dtype: object
Decoded sentence:  il est presque temps _END
-
Input sentence: 20122    You're very timid.
Name: eng, dtype: object
Decoded sentence:  vous êtes très timide _END
-
Input sentence: 40035    You don't frighten me.
Name: eng, dtype: object
Decoded sentence:  vous ne me faites pas peur _END
-
Input sentence: 40064    You had better go now.
Name: eng, dtype: object
Decoded sentence:  tu ferais mieux dy aller maintenant _END
-
Input sentence: 40056    You dropped something.
Name: eng, dtype: object
Decoded sentence:  vous avez laissé tomber quelque chose _END
-
Input sentence: 40068    You have gone too far.
Name: eng, dtype: object
Decoded sentence:  vous êtes trop loin _END
-
Input sentence: 40090    You heard your mother.
Name: eng, dtype: object
Decoded sentence:  vous avez entendu de le mère _END
-
Input sentence: 40095    You live too far away.
Name: eng, dtype: object
Decoded sentence:  vous êtes trop loin _END
-
In